# 1. load library

In [8]:
from datasets import load_dataset
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# 2. Load dataset

In [9]:
dataset = load_dataset("mabzak/anime_subtitle")

# 3. get dataset dan convert ke dataframe

In [10]:
train_data = dataset['train']

# Get kolom "Text Indonesia" dari train_data
text_indonesia = [data['Text Indonesia'] for data in train_data]

# Buat dataframe
df = pd.DataFrame(text_indonesia, columns=["Text Indonesia"])

print(df.head())

                                      Text Indonesia
0                          Tidak, akulah yang salah.
1                           Istriku tak selemah itu.
2  Kira-kira apa yang terjadi di antara mereka be...
3                    Nanti saja kupikirkan semuanya.
4             Hanya itulah yang terpenting sekarang!


# 4. Data Preprocessing

## 4.1 Lemmatization

In [26]:
# Inisialisasi Stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['Text Indonesia'] = df['Text Indonesia'].apply(lambda x: stemmer.stem(x))

## 4.2 Check Duplicates

In [27]:
# jumlah duplikat sebelum dihapus
duplicate_count = df.duplicated().sum()
print(f"Jumlah duplikat sebelum dihapus: {duplicate_count}")

Jumlah duplikat sebelum dihapus: 388


In [28]:
# Menghapus duplikat
df = df.drop_duplicates().reset_index(drop=True)

# Cetak data setelah menghapus duplikat
print(df)

                                    Text Indonesia  \
0                             tidak aku yang salah   
1                              istri tak lemah itu   
2       kira apa yang jadi di antara mereka dua ya   
3                           nanti saja pikir semua   
4                  hanya itu yang penting sekarang   
...                                            ...   
9834  apa pun yang jadi akan buat kau aku diri ini   
9835      kita hanya perlu lewat uji tahap dua ini   
9836        dan buka jalan untuk jadi orang hokage   
9837              kau tak perlu ingat hal itu pada   
9838                    alas untuk tak boleh kalah   

                                   Text Lemmatized  
0                             tidak aku yang salah  
1                              istri tak lemah itu  
2       kira apa yang jadi di antara mereka dua ya  
3                           nanti saja pikir semua  
4                  hanya itu yang penting sekarang  
...                              

# 5. Modelling

## 5.1 Tokenization

In [29]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text Indonesia'])

# Konversi teks menjadi sequence of integers
sequences = tokenizer.texts_to_sequences(df['Text Indonesia'])

# Padding sequences
maxlen = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=maxlen)

## 5.2 Create Model

In [33]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=maxlen))
model.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(LSTM(64, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))


## 5.3 train model

In [34]:
y = tokenizer.texts_to_matrix(df['Text Indonesia'], mode='binary')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model dengan callback
model.fit(X, y, epochs=20, batch_size=32, callbacks=[reduce_lr], verbose=1)


Epoch 1/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0635 - loss: 116.7723 - learning_rate: 0.0010
Epoch 2/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0743 - loss: 1230.2529 - learning_rate: 0.0010
Epoch 3/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0619 - loss: 3425.5718 - learning_rate: 0.0010
Epoch 4/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0635 - loss: 6206.3218 - learning_rate: 0.0010
Epoch 5/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0502 - loss: 9440.4062 - learning_rate: 0.0010
Epoch 6/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0558 - loss: 13114.8086 - learning_rate: 0.0010
Epoch 7/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0510 - loss: 16446.2266 - learning_rate: 0.0010
Epoch 8/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0482 - loss: 20121.5977 - learning_rate: 0.0010
Epoch 9/20
308/308 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0466 - loss: 23754.5664 - le

# 6. Test

In [38]:
# Input dari pengguna
test_input = input("Tanya: ")

# Lemmatization
test_input_lemmatized = stemmer.stem(test_input)

# Tokenization
test_input_seq = tokenizer.texts_to_sequences([test_input_lemmatized])
test_input_pad = pad_sequences(test_input_seq, maxlen=maxlen)

# Prediksi
predicted = model.predict(test_input_pad)

# Konversi hasil prediksi kembali ke teks
predicted_text = tokenizer.sequences_to_texts([predicted.argmax(axis=1)])[0]

# Output
print(f"Jawab: {predicted_text}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Jawab: itu
